In [1]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from django.db.models import Count, Avg
from multiprocess import Pool
from functools import partial
import numpy as np
from functools import partial
from scipy.sparse import coo_matrix, csr_matrix, find, tril
import networkx as nx
import csv

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.models import *
qid = 1457

In [2]:
runs = [282, 283, 284, 285, 287]


fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname)

for i in range(1,len(runs)):

    topics1 = DynamicTopic.objects.filter(run_id=runs[i-1])
    topics2 = DynamicTopic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [3]:
runs = list(range(295,300))

writer = pd.ExcelWriter(fname)
fname = "../../data/topic_data/run_{}.xlsx".format(runs[0])

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])

    df = pd.DataFrame.from_dict(list(topics2.values('title','id')))
    df['similarity'] = 0
    df['stopic'] = ""

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df['id']==t.id,'similarity'] = m
        df.loc[df['id']==t.id,'stopic'] = titles[scores.index(max(scores))]


    df = df.sort_values('similarity',ascending=False)
    df.head()
    
    
    
    
    

    
    df.to_excel(writer,"run_{}".format(runs[i]))
    
writer.save()

In [10]:
runs = list(range(295,300))

#runs = [295,296,297]

col1s = []
col2s = []
ss = []

for i in range(1,len(runs)):

    topics1 = Topic.objects.filter(run_id=runs[i-1])
    topics2 = Topic.objects.filter(run_id=runs[i])
    

    df = pd.DataFrame.from_dict(list(topics2.values('title')))
    
    df2 = pd.DataFrame.from_dict([{'title': 'None'}])
    
    df = df.append(df2)

    
    
    col1 = "topics_{}".format(runs[i-1])
    
    col1s.append(col1)
    
    col2 = "topics_{}".format(runs[i])
    
    col2s.append(col2) 
    
    s = "similarity_{}".format(i)
    ss.append(s)
    
    df = df.rename(columns = {'title': col2})

    df[s] = 0
    df[col1] = "None"

    for t in topics2:
        scores = [0]
        titles = [""]
        for ct in topics1:
            score = len(set(t.top_words).intersection(set(ct.top_words)))
            if score>0:
                scores.append(score)
                titles.append(ct.title)

        
        m = max(scores)
        df.loc[df[col2]==t.title, s] = m
        if m==0:
            df.loc[df[col2]==t.title, col1] = 'None'
        else:
            df.loc[df[col2]==t.title, col1] = titles[scores.index(max(scores))]

    if i==1:   
        #df = pd.DataFrame.from_dict(list(topics2.values('title')))
        mdf = df
    else:
        mdf = mdf.merge(df,how="outer").fillna("")
    
    

mdf.head(25)



,topics_296,similarity_1,topics_295,topics_297,similarity_2,topics_298,similarity_3,topics_299,similarity_4
0,"{research, review, develop}",4,"{research, social, develop}","{research, review, literatur}",8,"{research, review, literatur}",8,"{review, discuss, challeng}",5
1,"{research, review, develop}",4,"{research, social, develop}","{research, review, literatur}",8,"{research, review, literatur}",8,"{research, knowledg, scienc}",5
2,"{research, review, develop}",4,"{research, social, develop}","{social, econom, sustain}",2,"{social, econom, dimens}",5,"{social, econom, sustain}",6
3,"{energi, renew, consumpt}",8,"{energi, renew, consumpt}","{energi, renew, consumpt}",9,"{effici, improv, exergi}",1,"{effici, improv, exergi}",10
4,"{energi, renew, consumpt}",8,"{energi, renew, consumpt}","{energi, renew, consumpt}",9,"{energi, renew, sourc}",8,"{renew, sourc, generat}",4
5,"{energi, renew, consumpt}",8,"{energi, renew, consumpt}","{energi, renew, consumpt}",9,"{energi, renew, sourc}",8,"{energi, consumpt, save}",7
6,"{irrig, salin, agricultur}",4,"{crop, agricultur, yield}","{irrig, salin, rice}",7,"{irrig, salin, yield}",7,"{irrig, salin, yield}",10
7,"{water, resourc, basin}",7,"{water, resourc, irrig}","{water, basin, river}",8,"{water, resourc, qualiti}",7,"{water, drink, suppli}",9
8,"{water, resourc, basin}",7,"{water, resourc, irrig}","{water, basin, river}",8,"{river, basin, sediment}",3,"{river, basin, sediment}",10
9,"{urban, citi, plan}",8,"{urban, citi, plan}","{transport, public, vehicl}",2,"{transport, public, vehicl}",10,"{transport, public, travel}",8


In [11]:
columns = []
for i in range(len(col1s)):
    columns.append(col1s[i])
    columns.append(ss[i])
    if i == len(col1s)-1:
        columns.append(col2s[i])

print(columns)

mdf = mdf[columns]
mdf.head()

['topics_295', 'similarity_1', 'topics_296', 'similarity_2', 'topics_297', 'similarity_3', 'topics_298', 'similarity_4', 'topics_299']


,topics_295,similarity_1,topics_296,similarity_2,topics_297,similarity_3,topics_298,similarity_4,topics_299
0,"{research, social, develop}",4,"{research, review, develop}",8,"{research, review, literatur}",8,"{research, review, literatur}",5,"{review, discuss, challeng}"
1,"{research, social, develop}",4,"{research, review, develop}",8,"{research, review, literatur}",8,"{research, review, literatur}",5,"{research, knowledg, scienc}"
2,"{research, social, develop}",4,"{research, review, develop}",2,"{social, econom, sustain}",5,"{social, econom, dimens}",6,"{social, econom, sustain}"
3,"{energi, renew, consumpt}",8,"{energi, renew, consumpt}",9,"{energi, renew, consumpt}",1,"{effici, improv, exergi}",10,"{effici, improv, exergi}"
4,"{energi, renew, consumpt}",8,"{energi, renew, consumpt}",9,"{energi, renew, consumpt}",8,"{energi, renew, sourc}",4,"{renew, sourc, generat}"


In [16]:
#col1s.append(col2)


#mdf = mdf.sort_values(columns,ascending=False).reset_index()


res = mdf.groupby(columns)



res = res.apply(lambda x: x.sort_values(s,ascending=False))

l = len(res)




fname = "../../data/topic_data/run_compare_{}.xlsx".format(runs[0])

writer = pd.ExcelWriter(fname, engine='xlsxwriter')

res.to_excel(writer)

worksheet = writer.sheets['Sheet1']

for i in range(len(ss)):
    c = chr(ord('A')-1+(i+1)*2)
    r = "{}2:{}{}".format(c,c,len(res))
    print(r)
    
    worksheet.conditional_format(r, {'type': '3_color_scale'})

writer.save()

res.head() 
    
    
res.head()

B2:B131
D2:D131
F2:F131
H2:H131


topics_295  \
topics_295               similarity_1 topics_296                  similarity_2 topics_297                  similarity_3 topics_298                   similarity_4 topics_299                                                   
None                     0            None                        0            None                        0.0          None                         0.0          None                         130                      None   
                                                                                                                                                                  }                            129                      None   
{build, design, thermal} 4            {heat, thermal, temperatur} 9            {heat, thermal, temperatur} 3.0          {temperatur, degre, thermal} 9.0          {temperatur, degre, increas} 46   {build, design, thermal}   
                                                                                                           7.0          {heat, thermal, cool}        3.0          {thermal, comfort, insul}    44   {build, design, thermal}   
                                                                                                                                                     7.0          {heat, cool, transfer}       45   {build, design, thermal}   

                                                                                                                                                                                                    similarity_1  \
topics_295               similarity_1 topics_296                  similarity_2 topics_297                  similarity_3 topics_298                   similarity_4 topics_299                                       
None                     0            None                        0            None                        0.0          None                         0.0          None                         130             0   
                                                                                                                                                                  }                            129             0   
{build, design, thermal} 4            {heat, thermal, temperatur} 9            {heat, thermal, temperatur} 3.0          {temperatur, degre, thermal} 9.0          {temperatur, degre, increas} 46              4   
                                                                                                           7.0          {heat, thermal, cool}        3.0          {thermal, comfort, insul}    44              4   
                                                                                                                                                     7.0          {heat, cool, transfer}       45              4   

                                                                                                                                                                                                                     topics_296  \
topics_295               similarity_1 topics_296                  similarity_2 topics_297                  similarity_3 topics_298                   similarity_4 topics_299                                                      
None                     0            None                        0            None                        0.0          None                         0.0          None                         130                         None   
                                                                                                                                                                  }                            129                         None   
{build, design, thermal} 4            {heat, thermal, temperatur} 9            {heat, thermal, temperatur} 3.0          {temperatur, degre, thermal} 9.0          {temperatur, degre, increas} 46   {heat, thermal, temperatur}   
                     

In [ ]:
print(mdf)

In [ ]:
runs = list(range(295,300))

runs = []

for i in range(len(runs)):

    topics = Topic.objects.filter(run_id=runs[i])
    
    tree = []
    
    for t in topics:
        
        tdict = {
            "parent_similarity": 0,
            "run": runs[i],
            "ntopics": topics.count(),
            "title": t.title,
            "sim_topics": [],
            "child_topics": 0
        }
        
        if i==0:
            tree.append(tdict)
        else:
            
            scores = [0]
            titles = [""]
            for ct in comparison_topics:
                score = len(set(t.top_words).intersection(set(ct.top_words)))
                if score>0:
                    scores.append(score)
                    titles.append(ct.title)


                parent = titles[scores.index(max(scores))]
                
                
                
                print(m)

In [ ]:
def find_similar(x):
    return(1,2)

df['bla'], df['x'] = zip(df.map(find_similar))
                         
df.head()